In [1]:
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.wait import WebDriverWait
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
import pendulum
import operator as op

driver = uc.Chrome()
time.sleep(3)
driver.get('https://www.indeed.com/')
time.sleep(6)

In [94]:
keyword_filter= "data analyst"
location_filter = "vietnam"
page_count_filter = "1"
salary_filter = "3000000 VNĐ month"
date_posted_filter = "2020-02-02 02:00:00"
time.sleep(3)

In [3]:
search_job_keyword = driver.find_element(By.XPATH, '//input[@aria-label="search: Job title, keywords, or company"]')
time.sleep(3)
search_job_keyword.click()
time.sleep(3)
search_job_keyword.send_keys(keyword_filter)
time.sleep(3)
search_location_keyword = driver.find_element(By.XPATH, '//input[@aria-label="Edit location"]')
time.sleep(3)
search_location_keyword.click()
time.sleep(3)
search_location_keyword.send_keys(location_filter)
time.sleep(3)
search_button = driver.find_element(By.XPATH, '//button[@class="yosegi-InlineWhatWhere-primaryButton"]')
time.sleep(3)
search_button.submit()
time.sleep(6)

In [4]:
def search_job_indeed():
    global search_job_indeed_list, search_job_indeed_final_list, search_job_indeed_final_list_filter
    search_job_indeed_final_list = []
    time.sleep(3)
    while True:
        try:
            job_indeed_frame = driver.find_elements(By.XPATH, '//div[@id="mosaic-provider-jobcards"]/ul/li/div/div/div/div/div/div/table/tbody/tr/td[1]/div[1]/h2/a')
            time.sleep(3)
            search_job_indeed_list = list(set((job_indeed.get_attribute('href') for job_indeed in job_indeed_frame)))
            time.sleep(3)
            job_indeed_frame[-1].location_once_scrolled_into_view
            time.sleep(6)
        except:
            print('No more job')
            time.sleep(3)
        search_job_indeed_final_list.extend(search_job_indeed_list)
        time.sleep(3)
        search_job_indeed_final_list = list(set(search_job_indeed_final_list))
        time.sleep(3)
        if (len(search_job_indeed_final_list) % 15 == 0 and len(search_job_indeed_final_list) // 15 == int(page_count_filter)) or (len(search_job_indeed_final_list) % 15 != 0 and len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter)):
            break
        try:
            next_page_button = driver.find_elements(By.XPATH, '//nav[@aria-label="pagination"]/ul/li/a')
            time.sleep(3)
            next_page_button[-1].click()
            time.sleep(3)
            yield len(search_job_indeed_final_list)
        except:
            print('No more page')
            time.sleep(3)
            break

generators = search_job_indeed()
for generator in generators:
    print(generator)

In [5]:
if len(search_job_indeed_final_list) % 15 == 0:
    if len(search_job_indeed_final_list) == 0:
        search_job_indeed_final_list_filter = []
    elif len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]
else:
    if len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    elif len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:(int(page_count_filter) - 1) * 15 + len(search_job_indeed_final_list) % 15]
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]

In [6]:
len(search_job_indeed_final_list_filter)

15

In [20]:
def get_job_indeed_description():
        global job_indeed_description_list
        job_indeed_description_list = []
        for _, job_indeed_url in enumerate(search_job_indeed_final_list_filter):
            driver.get(job_indeed_url)
            time.sleep(6)
            try:
                    company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                    time.sleep(3)
                    sub_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[2]/span').text
                    time.sleep(3)
                    if len(re.findall(r'\b\d+\.\d+\b', sub_company)) > 0:
                        total_company = company
                    else:
                        total_company = company + ' ' + sub_company
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                            time.sleep(3)
                    except:
                            try:
                                    time.sleep(3)
                                    total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span').text
                                    time.sleep(3)
                            except:
                                    try:
                                            time.sleep(3)
                                            total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                            time.sleep(3)
                                    except:
                                            try:
                                                    time.sleep(3)
                                                    total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                                    time.sleep(3)
                                            except:
                                                    time.sleep(3)
                                                    total_company = ''
                                                    time.sleep(3)
            try:
                    header = driver.find_element(By.XPATH, '//div[starts-with(@class,"jobsearch-JobInfoHeader-title-container")]/h1/span').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            header = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/span').text
                            time.sleep(3)
                    except:
                            time.sleep(3)
                            header = ''
                            time.sleep(3)
            try:
                    location = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div[2]/div').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/div[2]/div').text
                            time.sleep(3)
                    except:
                            try:
                                time.sleep(3)
                                location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div[2]/div/div/div/div[2]/div').text
                                time.sleep(3)
                            except:
                                    time.sleep(3)
                                    location = ''
                                    time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_context = soup.find_all('script', string=re.compile(r'@context'))[0].getText().strip()
                time.sleep(3)
                standard_date_posted = re.search(r'\"datePosted\":\"([^"]+)\"', script_context).group(1)
                time.sleep(3)
                readable_date_posted = pendulum.parse(standard_date_posted, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                time.sleep(3)
            except:
                try:
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    time.sleep(3)
                    script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                    time.sleep(3)
                    standard_date_posted = re.search(r'\"datePublished\":(\d+)', script_initial_data).group(1)
                    time.sleep(3)
                    readable_date_posted = pendulum.from_timestamp(int(standard_date_posted) / 1000, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                    time.sleep(3)
                except:
                        time.sleep(3)
                        readable_date_posted = ''
                        time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                time.sleep(3)
                salary = re.search(r'\"salaryText\":\"([^"]+)\"', script_initial_data).group(1)
                time.sleep(3)
            except:
                    time.sleep(3)
                    salary = ''
                    time.sleep(3)
            job_indeed_description_list.append((job_indeed_url, total_company, header, salary, location, readable_date_posted))
            yield total_company, header, location, readable_date_posted, salary
generators = get_job_indeed_description()
for generator in generators:
    print(generator)

('Mantu', 'Financial Data Analyst - F/M/X', 'Thành phố Hồ Chí Minh', '2025-01-06 10:26:51', '')
('Công ty TNHH Esuhai', 'Chuyên viên phân tích dữ liệu - Data analyst', 'Thành phố Hồ Chí Minh', '2022-02-24 10:11:01', '')
('Wolf Solutions', 'Data Analyst Intern', 'Thành phố Hồ Chí Minh', '2022-09-30 07:49:39', '')
('LEGO', 'Manufacturing Data Analyst', 'Bình Dương', '2025-01-07 07:27:56', '')
('Concung.com', 'Data Analyst - Business Intelligence', 'Thành phố Hồ Chí Minh', '2024-12-23 22:12:48', '')
('LARION Consulting & Software Development Joint Stock Company', 'Business Analyst Intern', 'Thành phố Hồ Chí Minh', '2024-11-14 07:28:53', '1,000,000 VNĐ - 3,000,000 VNĐ a month')
('Chứng Khoán Thiên Việt', 'DATA ANALYST – DATA TEAM', 'Thành phố Hồ Chí Minh', '2023-06-29 14:28:25', '')
('VNG', 'Associate Data Analyst, Zalopay', 'Thành phố Hồ Chí Minh', '2024-08-19 10:03:21', '')
('Công TY TNHH Un - Available', 'Data Analytics Intern', 'Thành phố Hồ Chí Minh', '2025-01-04 13:41:52', '')
('LEGO

In [21]:
len(job_indeed_description_list)

15

In [24]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
import pandas as pd

credentials = service_account.Credentials.from_service_account_file('credentials.json')
spreadsheet_service = build('sheets', 'v4', credentials=credentials)

SPREADSHEET_ID = '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg'
RANGE_NAME = 'Sheet1'
sheet = spreadsheet_service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])
max_cols = max(len(row) for row in values)
values = [row + [''] * (max_cols - len(row)) for row in values]
df = pd.DataFrame(values[1:], columns=values[0])
df = df.fillna('')

In [26]:
df1 = pd.DataFrame(job_indeed_description_list, columns=df.columns).__deepcopy__()
df1

,Link,Company,Title,Salary,Location,Date posted
0,https://vn.indeed.com/rc/clk?jk=dc0463742d6a8b...,Mantu,Financial Data Analyst - F/M/X,,Thành phố Hồ Chí Minh,2025-01-06 10:26:51
1,https://vn.indeed.com/rc/clk?jk=ef6e87670a5b5c...,Công ty TNHH Esuhai,Chuyên viên phân tích dữ liệu - Data analyst,,Thành phố Hồ Chí Minh,2022-02-24 10:11:01
2,https://vn.indeed.com/rc/clk?jk=3178766f214cc3...,Wolf Solutions,Data Analyst Intern,,Thành phố Hồ Chí Minh,2022-09-30 07:49:39
3,https://vn.indeed.com/rc/clk?jk=3e45f07d403c59...,LEGO,Manufacturing Data Analyst,,Bình Dương,2025-01-07 07:27:56
4,https://vn.indeed.com/rc/clk?jk=0650c2efd27fc0...,Concung.com,Data Analyst - Business Intelligence,,Thành phố Hồ Chí Minh,2024-12-23 22:12:48
5,https://vn.indeed.com/rc/clk?jk=ded399353790bb...,LARION Consulting & Software Development Joint...,Business Analyst Intern,"1,000,000 VNĐ - 3,000,000 VNĐ a month",Thành phố Hồ Chí Minh,2024-11-14 07:28:53
6,https://vn.indeed.com/rc/clk?jk=18b09bde8afda6...,Chứng Khoán Thiên Việt,DATA ANALYST – DATA TEAM,,Thành phố Hồ Chí Minh,2023-06-29 14:28:25
7,https://vn.indeed.com/rc/clk?jk=2f9bfb933381a3...,VNG,"Associate Data Analyst, Zalopay",,Thành phố Hồ Chí Minh,2024-08-19 10:03:21
8,https://vn.indeed.com/rc/clk?jk=b8592fed541177...,Công TY TNHH Un - Available,Data Analytics Intern,,Thành phố Hồ Chí Minh,2025-01-04 13:41:52
9,https://vn.indeed.com/rc/clk?jk=8643b1e068402c...,LEGO,Business Analyst,,Bình Dương,2025-01-06 13:27:59


In [154]:
re.findall(r'(\d+\,?\d*)', df1.at[5, 'Salary'])

['1,000', '000', '3,000', '000']

In [118]:
def implement_salary_filter():
    global implement_filter_list
    implement_filter_list = []
    salary_filter_tuple = (salary_filter.split(' ')[0], salary_filter.split(' ')[1], salary_filter.split(' ')[2])
    for index, row in df1.iterrows():
        if op.contains(df1.at[index, 'Salary'], salary_filter_tuple[1]) and op.contains(df1.at[index, 'Salary'], salary_filter_tuple[2]):
            implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
            yield from implement_filter_list
salary_generators = implement_salary_filter()
for generator in salary_generators:
    print(generator)

('https://vn.indeed.com/rc/clk?jk=ded399353790bb22&bb=h5wsJqunwfLSrt-SgmHrAxTzERMIzt4Ftij6M-PbUa2DuHbdhSbHNPaJsL0NOTKO-iI2SlD7MQQ7rEzhbUdxf61A3usntt67vXLAQU4IY1vgG0hocqxC3w6Gsqt7S-18&xkcb=SoDZ67M33hirZ_wE6B0MbzkdCdPP&fccid=7bc0d9afbff1dc25&cmp=LARION&ti=Business+Intern&vjs=3', 'LARION Consulting & Software Development Joint Stock Company', 'Business Analyst Intern', '1,000,000 VNĐ - 3,000,000 VNĐ a month', 'Thành phố Hồ Chí Minh', '2024-11-14 07:28:53')
